# Musterlösung zur Aufgabe „Mastermind“

Diese Musterlösung enthält einen *Code Maker* und einen (allerdings nicht optimalen) *Code Breaker*.

In [3]:
import random

COLORS = ( 'r', 'g', 'b', 'o', 'w', 's')
COUNT = 4

# Scoring

Sowohl der *Code Maker* als auch der *Code Breaker* benötigen eine Funktion zur Bewertung eines Versuchs. 

In [35]:
def score(guess, secret):
    """ Evaluate a guess against the secret and return a tuple 
    containing the number of black and white pins. 
      
    The secret and guess remain unchanged. 
    
    >>> score(list("bbbb"), list("brrb"))
    (2, 0)
    >>> score("rbbr", "brrb")
    (0, 4)
    >>> score("rbrb", "brrb")
    (2, 2)
    """
    
    count = len(secret)
    black, white = (0, 0)
    
    # count black pins
    black = len({ i for i in range(count) if guess[i] == secret[i] })

    white = 0
    # count black + white pins
    for c in COLORS:
        white += min(guess.count(c), secret.count(c))
    white -= black
        
    return (black, white)

## Testen mit `doctest`

Die merkwürdigen Kommentarzeilen sind Testfälle, die wir wie folgt aufrufen können:

In [36]:
import doctest
doctest.testmod(verbose=True)

Trying:
    maker = CodeMaker()
Expecting nothing
ok
Trying:
    len(maker.get_secret()) == COUNT
Expecting:
    True
ok
Trying:
    maker = CodeMaker()
Expecting nothing
ok
Trying:
    maker.score(maker.secret)
Expecting:
    (4, 0)
ok
Trying:
    score(list("bbbb"), list("brrb"))
Expecting:
    (2, 0)
ok
Trying:
    score("rbbr", "brrb")
Expecting:
    (0, 4)
ok
Trying:
    score("rbrb", "brrb")
Expecting:
    (2, 2)
ok
7 items had no tests:
    __main__
    __main__.CodeBreaker
    __main__.CodeBreaker.__init__
    __main__.CodeBreaker.play
    __main__.CodeMaker
    __main__.CodeMaker.__init__
    __main__.scorer
3 items passed all tests:
   2 tests in __main__.CodeMaker.get_secret
   2 tests in __main__.CodeMaker.score
   3 tests in __main__.score
7 tests in 10 items.
7 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=7)

## Code Maker

Der *Code Maker* besteht im wesentlichen aus einer Methode zur Generierung des Codes und einer zur Bewertung.

In [37]:
class CodeMaker:
    """ The code maker part: Choose a secret code and evaluate guesses. """

    def __init__(self, count=COUNT, colors=COLORS):
        self.count = count
        self.colors = list(colors)
        self.secret = self.get_secret()

    def get_secret(self):
        """ Choose the secret code 
        
        >>> maker = CodeMaker()
        >>> len(maker.get_secret()) == COUNT
        True
        """
        secret = tuple(random.choice(self.colors) for i in range(self.count) )
        return secret

    def score(self, guess):
        """ Evaluate a guess 
        
        >>> maker = CodeMaker()
        >>> maker.score(maker.secret)
        (4, 0)
        """
        return score(guess, self.secret)

In [38]:
#cm = CodeMaker()

## Code Breaker

Der *Code Breaker* benutzt einen sehr einfachen (und daher nicht optimalen) Algorithmus:
- Nehme die Menge aller möglichen Codes.
- Entferne die Codes, die mit den bisherigen Antworten des *Code Makers* im Widerspruch stehen. Dies sind die Codes, die im Vergleich mit den bisher geratenen Codes einen anderen Score ergeben als der vom *Code Maker* gegebene.
- Probiere aus dieser Menge einen zufälligen Code.

Ein besserer Algorithmus würde einen Code wählen, für den eine Antwort des *Code Makers* möglichst viel zusätzliche Information bringt.

In [41]:
ALL_CODES = { (a, b, c, d) for a  in COLORS for b in COLORS for c in COLORS for d in COLORS }

class CodeBreaker:
    """ The code breaker tries to guess the code. """

    def __init__(self, scorer, count=COUNT, colors=COLORS):
        """ Create a code breaker with a scorer function """
        self.count = count
        self.colors = list(colors)
        self.scorer = scorer

    def play(self): 
        """ Play a game using the scorer function.
        
        Returns the number of guesses needed to break the code.
        """
        guesses = []
        guesses.append(random.choice(list(ALL_CODES)))
        scores = { guesses[0]: scorer(guesses[0]) }
        if scores[guesses[0]][0] == self.count:
            return 1

        candidates = set(ALL_CODES) - set(guesses)
        while len(candidates) > 0:
            invalid = set()
            for candidate in candidates:
                # check whether the candidate is consistent with the guesses we made
                # a consistent candidate should reproduce the scores we already got

                for guess in guesses:
                    sc = score(guess, candidate)
                    if sc != scores[guess]:
                       # different score - the candidate is not consistent with what we know
                       invalid.add(candidate) 
                       break
             
            candidates -= invalid
            candidate = random.choice(list(candidates))

            guesses.append(candidate)
            scores[candidate] = scorer(candidate)
            if scores[candidate][0] == self.count:
                return len(guesses)

## Auswertung

Wir spielen nun 1000 Partien und werten aus, wie viele Versuche der *Code Breaker* benötigt hat.

In [45]:
from tqdm.notebook import tqdm

N = 5000
sum = 0
stat = dict()

for i in tqdm(range(N)):
    coder = CodeMaker()

    def scorer(guess):
        res = coder.score(guess)
        return res

    breaker = CodeBreaker(scorer)
    rounds = breaker.play()
    sum += rounds
    if rounds in stat:
        stat[rounds] += 1
    else:
        stat[rounds] = 1

for i in sorted(stat.keys()):
    print(f"guessed in {i}: {stat[i]}")
print(f"average guesses needed: {sum/N:.2f}")


guessed in 1: 2
guessed in 2: 59
guessed in 3: 343
guessed in 4: 1662
guessed in 5: 2226
guessed in 6: 659
guessed in 7: 49
average guesses needed: 4.64
